# How to add node retry policies

There are many use cases where you may wish for your node to have a custom retry policy, for example if you are making HTTP requests, calling an API, etc. 

## Defining your retry policy

In order to configure the retry policty, you have to pass the `retry` parameter to the `add_node` function. The `retry` parameter takes in a `RetryPolicy` named tuple object. Below we instantiate a `RetryPolicy` object with the default parameters.

In [8]:
from langgraph.pregel.types import default_retry_on
from langgraph.pregel import RetryPolicy

# Note: This is equivalent to writing retry_policy=RetryPolicy() since these are all default values
retry_policy = RetryPolicy(
    initial_interval = 0.5,
    backoff_factor = 2.0,
    max_interval = 128.0,
    max_attempts = 3,
    jitter = True,
    retry_on = default_retry_on
)

You can then change the default values to work for your custom use case:

In [9]:
retry_policy = retry_policy._replace(max_attempts = 5)
retry_policy

RetryPolicy(initial_interval=0.5, backoff_factor=2.0, max_interval=128.0, max_attempts=5, jitter=True, retry_on=<function default_retry_on at 0x1157419e0>)

If you want more information on what each of the parameters does, be sure to read the source code.

## Passing your retry policy to a node

Lastly, we can pass `retry_policy` to the `add_node` function:

In [10]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import BaseMessage

from langgraph.graph import END, StateGraph, START


model = ChatAnthropic(model_name="claude-2.1")


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


def _call_model(state):
    response = model.invoke(state["messages"])
    return {"messages": [response]}

# Define a new graph
workflow = StateGraph(AgentState)
workflow.add_node("model", _call_model,retry=retry_policy)
workflow.add_edge(START, "model")
workflow.add_edge("model", END)

app = workflow.compile()

Now the `"model"` node will be configured to use the custom retry policy that was defined above.